# IntroStat Week 9

Welcome to the 9th lecture in IntroStat

During the lectures we will present both slides and notebooks. 

This is the notebook used in the lecture in week 9.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.power as smp


### Example: Ozon concentration

Today we will be using the same data for all examples

The **Airquality dataset**

In [ ]:
# Find the "airquality" dataset and print first 10 rows:
Air = sm.datasets.get_rdataset("airquality").data
print(Air.head(10))

Daily readings of the following air quality values for May 1, 1973 (a Tuesday) to September 30, 1973:

- Ozone: Mean ozone in parts per billion from 1300 to 1500 hours at Roosevelt Island

- Solar.R: Solar radiation in Langleys in the frequency band 4000–7700 Angstroms from 0800 to 1200 hours at Central Park

- Wind: Average wind speed in miles per hour at 0700 and 1000 hours at LaGuardia Airport

- Temp: Maximum daily temperature in degrees Fahrenheit at La Guardia Airport.

Notice the dataset has some missing values (NaN)

In [ ]:
# How many rows? (obs: number of observations is smaller due to missing values in dataset)
print(len(Air))

In [ ]:
# The dataset is now a *Pandas Dataframe*
print(type(Air))

When working with "real" data one typically need to do some *data-wrangling* 

This may include joining data from different tables, re-ordering data, re-structuring the dataset

Today we need to rename a column:

In [ ]:
# rename "Solar.R", because the use of "." means something else in python and this could give rise to errors:
Air = Air.rename(columns={"Solar.R": "SolarR"})
print(Air.head(10))

#### We are interested in the Ozone values 

How does Ozone depend on solar radiation, wind and temperature?

In [ ]:
# Visualise the data

# split to make 3 plots:
fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(16,4))

# scatter plots with Ozone versus the three other columns
Air.plot.scatter('Wind',    'Ozone', ax=ax0)
Air.plot.scatter('SolarR',  'Ozone', ax=ax1)
Air.plot.scatter('Temp',    'Ozone', ax=ax2)

The second plot seems to have variance increasing with increasing solar radiation

The dataset seems to have more Ozone values that are relatively small and fewer values that are relatively large

We will now inspect the distribution of Ozone values (and consider whether a data-transformation a appropriate)

In [ ]:
# Do a histogram of "Ozone" column
plt.hist(Air["Ozone"], density=True)
plt.show()

In [ ]:
# Since the distribution is very skewed we will do a data-transformation and compute the logarithm of the Ozone column:
Air["logOzone"] = np.log(Air["Ozone"])   # this creates an axtra column in the dataframe
print(Air)

plt.hist(Air["logOzone"], density=True)
plt.show()

This distribution is nicer (but maybe not perfect - the value with logOzone=0 could be considered an outlier)

In [ ]:
# Visualise the transformed data

# split to make 3 plots:
fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(16,4))

# scatter plots with logOzone versus the three other columns
Air.plot.scatter('Wind',    'logOzone', ax=ax0)
Air.plot.scatter('SolarR',  'logOzone', ax=ax1)
Air.plot.scatter('Temp',    'logOzone', ax=ax2)

Considerations:

The datapoint with logOzone=0 could be an outlier - maybe we should remove it from the dataset.

We could model logOzone as a linear function of either wind, solar radiation or temperature. 

In [ ]:
# remove the outlier:
Air = Air[(Air["logOzone"] != 0)]

In [ ]:
print(len(Air))

After removing outlier we have **152 rows** in the dataset

#### Now we do **Simpel Linear Regression** with one explanatory variable at the time

In [ ]:
# logOzone and temperature:
fit_temp = smf.ols(formula = 'logOzone ~ Temp', data=Air).fit()
print(fit_temp.summary(slim=True))

### **>>> Kahoot 1 (3 questions)**

In [ ]:
# Get predictions based on the model:
temp_range = pd.DataFrame({'Temp': np.arange(50,110, 10)})
print(temp_range)

pred = fit_temp.get_prediction(temp_range).summary_frame(alpha=0.05)
print(pred.head())

### **>>> Kahoot 2 (1 questions)**

In [ ]:
# Plot data with model predictions:
Air.plot.scatter('Temp',    'logOzone')
plt.plot(temp_range, pred["mean"], color="red")
plt.scatter(temp_range, pred["mean"], color="red")
plt.show()

Nice - this is a *simple linear regression model* with temperature as explanatory variable (and logOzone as dependent variable / outcome variable)

Lets try another explanatory variable:

In [ ]:
# logOzone verus solar radiation:

# fit the model
fit_rad = smf.ols(formula = 'logOzone ~ SolarR', data=Air).fit()
print(fit_rad.summary(slim=True))

# Get prediction based on the model:
rad_range = pd.DataFrame({'SolarR': np.arange(0,360, 50)})
pred = fit_rad.get_prediction(rad_range).summary_frame(alpha=0.05)

# Plot data with model:
Air.plot.scatter('SolarR',    'logOzone')
plt.plot(rad_range, pred["mean"], color="red")
plt.scatter(rad_range, pred["mean"], color="red")
plt.show()


Looks good (?)

Now we try the last explanatory variable:

In [ ]:
# logOzone verus wind:

# fit the model
fit_wind = smf.ols(formula = 'logOzone ~ Wind', data=Air).fit()
print(fit_wind.summary(slim=True))

# Get prediction based on the model:
wind_range = pd.DataFrame({'Wind': np.arange(0,30,5)})
pred = fit_wind.get_prediction(wind_range).summary_frame(alpha=0.05)

# Plot data with model predictions:
Air.plot.scatter('Wind',    'logOzone')
plt.plot(wind_range, pred["mean"], color="red")
plt.scatter(wind_range, pred["mean"], color="red")
plt.show()

#### Can we somehow combine all the variables into one model?? 

#### The answer is yes - with **"Multiple Linear Regression"** 

(return to slides/blackboard before proceding)

### Example: Ozon concentration with Multiple Linear Regression

In [ ]:
# Mulitiple Linear Regression

# fit the full model with 3 explanatopry variables:
fit_full_model = smf.ols(formula = 'logOzone ~ Temp + SolarR + Wind', data=Air).fit()  # Notice how the model is formulated in python!
print(fit_full_model.summary(slim=True))

There is a coefficient (parameter estimate) for each explanatory variable in the model (+ for the intercept)

Each parameter estimate also has a Standard Error

For each parameter a t-test is performed (t_obs and p-values and confidence intervals are calculated)

### Example: Confidence and prediction intervals

In [ ]:
x_new    = pd.DataFrame({'Temp':   [70],   # a bit of "wrangling" is needed
                         'SolarR': [200],
                         'Wind':   [10]})
print(x_new)

In [ ]:
pred = fit_full_model.get_prediction(x_new).summary_frame(alpha=0.05)    # get predictions
print(pred.head(5))

Now we have predicted outcome for given values of explanatory variables 

(Make sure you understand the meaning of all the printed values above)

In [ ]:
# We will now predict for MANY values of Temp and Wind (but keep SolarR constant in order to be able to plot in 3D)

Temp_range = np.linspace(50, 100, 100)
Wind_range = np.linspace(0, 25, 100)
SolarR_new = Air["SolarR"].mean()

Temp_new, Wind_new = np.meshgrid(Temp_range, Wind_range)        # make a meshgrid

In [ ]:
New_data = pd.DataFrame({'Temp':   Temp_new.reshape(1,-1)[0],   # a bit of "wrangling" is needed
                         'SolarR': SolarR_new,
                         'Wind':   Wind_new.reshape(1,-1)[0]})
print(New_data)

In [ ]:
pred = fit_full_model.get_prediction(New_data).summary_frame(alpha=0.05)    # get predictions
print(pred.head(5))

In [ ]:
New_data_with_pred = pd.concat([New_data, pred], axis=1)                    # merge "x"-data with predictions
print(New_data_with_pred.head(5))
print(len(New_data_with_pred))

In [ ]:
# Create a 3D plot 
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot a surface
ax.plot_surface(np.array(New_data_with_pred["Temp"]).reshape(100,100),
                np.array(New_data_with_pred["Wind"]).reshape(100,100),
                np.array(New_data_with_pred["mean"]).reshape(100,100))

ax.plot_surface(np.array(New_data_with_pred["Temp"]).reshape(100,100),
                np.array(New_data_with_pred["Wind"]).reshape(100,100),
                np.array(New_data_with_pred["mean_ci_upper"]).reshape(100,100), color="grey")

ax.plot_surface(np.array(New_data_with_pred["Temp"]).reshape(100,100),
                np.array(New_data_with_pred["Wind"]).reshape(100,100),
                np.array(New_data_with_pred["mean_ci_lower"]).reshape(100,100), color="grey")

ax.plot_surface(np.array(New_data_with_pred["Temp"]).reshape(100,100),
                np.array(New_data_with_pred["Wind"]).reshape(100,100),
                np.array(New_data_with_pred["obs_ci_upper"]).reshape(100,100), color="lightgrey")

ax.plot_surface(np.array(New_data_with_pred["Temp"]).reshape(100,100),
                np.array(New_data_with_pred["Wind"]).reshape(100,100),
                np.array(New_data_with_pred["obs_ci_lower"]).reshape(100,100), color="lightgrey")

# Add labels
ax.set_xlabel('Temp')
ax.set_ylabel('Wind')
ax.set_zlabel('logOzone')

ax.view_init(elev=30, azim=20)
#ax.view_init(elev=20, azim=10)




plt.show()

The blue plane are the predicted values. 

The light grey is the prediction interval.

The dark grey is the confidence-interval (for the plane).

(back to slides)

### Example: Model control


In [ ]:
# full model with 3 explanatory variables:
fit_full_model = smf.ols(formula = 'logOzone ~ Temp + SolarR + Wind', data=Air).fit()

# residuals:
residuals = fit_full_model.resid
fittedvalues = fit_full_model.fittedvalues

In [ ]:
# Plot residuals
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(12,4))

# qq-plot of resiudals:
sm.qqplot(residuals,ax=ax1, line='s')
ax1.set_title("QQ plot of residuals")

# plot residuals versus fitted values:
ax2.scatter(fittedvalues,residuals)
ax2.set_xlabel("fitted values")
ax2.set_ylabel("residuals")

plt.tight_layout()
plt.show()

qq-plot: check normality of residuals (this one looks fine because the points are almost on the straight line)


scatter plot: is used to look for patterns in residuals (here we se no obvious pattern - at least not a strong one)

In [ ]:
# plot residuals versus individual explanatory variables:
Air["residuals"] = residuals
fig, (ax0, ax1, ax2) = plt.subplots(1,3,figsize=(12,4))

# plot residuals versus fitted values:
ax0.scatter(Air["Temp"],Air["residuals"])
ax0.set_xlabel("Temp")
ax0.set_ylabel("residuals")

ax1.scatter(Air["SolarR"],Air["residuals"])
ax1.set_xlabel("SolarR")
ax1.set_ylabel("residuals")

ax2.scatter(Air["Wind"],Air["residuals"])
ax2.set_xlabel("Wind")
ax2.set_ylabel("residuals")

plt.tight_layout()
plt.show()

Here we see no obvious patterns in the residuals - which is good!

## Example: curvilinear regression

In [ ]:
# Look at the original data (but with log(ozone))

# split to make 4 plots:
fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(16,4))

# scatter plots with logOzone versus the three other columns
Air.plot.scatter('Wind',    'logOzone', ax=ax0)
Air.plot.scatter('SolarR', 'logOzone', ax=ax1)
Air.plot.scatter('Temp',    'logOzone', ax=ax2)

Would it be a good idea to include some squared terms?

Lets try with **Wind**:

In [ ]:
# Linear model with wind (simple linear regression):

# fit the model
fit_wind = smf.ols(formula = 'logOzone ~ Wind', data=Air).fit()
print(fit_wind.summary(slim=True))

# Get prediction based on the model:
wind_range = pd.DataFrame({'Wind': np.arange(0,25.5,.5)})
pred = fit_wind.get_prediction(wind_range).summary_frame(alpha=0.05)

# Plot data with model predictions:
Air.plot.scatter('Wind',    'logOzone')
plt.plot(wind_range, pred["mean"], color="red")
plt.scatter(wind_range, pred["mean"], color="red")
plt.show()

In [ ]:
# make column with wind-squared /values:
Air["Wind2"] = Air["Wind"]**2

print(Air.head())

In [ ]:
# Model with Wind and Wind2

# fit the model
fit_wind2 = smf.ols(formula = 'logOzone ~ Wind + Wind2', data=Air).fit()
print(fit_wind2.summary(slim=True))

This looks great - wind-squared is also significant. 

Is it a better model than the linear one?

In [ ]:
# Visual inspection:

# Get prediction based on the model:
wind_range["Wind2"] = wind_range["Wind"]**2
pred2 = fit_wind2.get_prediction(wind_range).summary_frame(alpha=0.05)

# Plot data with model predictions:
Air.plot.scatter('Wind',    'logOzone')
plt.plot(wind_range["Wind"], pred2["mean"], color="red")
plt.scatter(wind_range["Wind"], pred2["mean"], color="red")
plt.show()

Is it a better model?

Is it an *"overfit"*?

In [ ]:
# lets have a quick look at the residuals (model control) in both linear and quadratic cases:
# residuals:
residuals = fit_wind.resid
fittedvalues = fit_wind.fittedvalues

residuals2 = fit_wind2.resid
fittedvalues2 = fit_wind2.fittedvalues


fig, axs = plt.subplots(2,3,figsize=(14,8))

# for linear model:
# qq-plot of resiudals:
sm.qqplot(residuals,ax=axs[0,0], line='s')
axs[0,0].set_title("QQ plot of residuals - Linear Model")

# plot residuals versus fitted values:
axs[0,1].scatter(fittedvalues,residuals)
axs[0,1].set_xlabel("fitted values")
axs[0,1].set_ylabel("residuals")
axs[0,1].set_title("Residuals - Linear Model")

# plot data with model
Air.plot.scatter('Wind',    'logOzone', ax = axs[0,2])
axs[0,2].plot(wind_range["Wind"], pred["mean"], color="red")

# for quadratic model:
# qq-plot of resiudals:
sm.qqplot(residuals2,ax=axs[1,0], line='s')
axs[1,0].set_title("QQ plot of residuals - Quadratic Model")

# plot residuals versus fitted values:
axs[1,1].scatter(fittedvalues2,residuals2)
axs[1,1].set_xlabel("fitted values")
axs[1,1].set_ylabel("residuals")
axs[1,1].set_title("Residuals - Quadratic Model")

# plot data with model
Air.plot.scatter('Wind',    'logOzone', ax = axs[1,2])
axs[1,2].plot(wind_range["Wind"], pred2["mean"], color="red")


plt.tight_layout()
plt.show()

Which model is better?

### **>>> Kahoot (1 question)**

Maybe it would make more sense to include a quadratic term for the SolarR-variable. You can try this yourself! (simply change Wind and Wind2 to SolarR and SolarR2 in the code above)

### Example: Colinearity and confounding

In [ ]:
# lets make a FAKE variable which is super CORRELATED TO WIND:
Air["fake_wind"] = 2*Air["Wind"] + stats.norm.rvs(size=len(Air), loc=0, scale=1)
print(Air.head())

In [ ]:
Air.plot.scatter('fake_wind',    'logOzone')

In [ ]:
Air.plot.scatter('fake_wind',    'Wind')

In [ ]:
# Now we fit a model using both Wind and fake_wind:
fit_full_model = smf.ols(formula = 'logOzone ~ Temp + SolarR + Wind + fake_wind', data=Air).fit()
print(fit_full_model.summary(slim=True))

Both Wind and fake_wind become insignificant!

### **>>> Kahoot (1 question)**

### Example: Model selection

In [ ]:
# re-visit the Multiple Linear Regression model:
print(fit_full_model.summary(slim=True))

Can we include even more explanatory variables?

Are more variables always better?

#### How do we select the best model? How do we choose which explanatory variables to include?

In [ ]:
# First imagine we had even MORE explanatory variables:

# lets make a "fake" variable (that is independent of the other variables):
Air["fake"] = stats.norm.rvs(size=len(Air))
print(Air.head())

Air.plot.scatter('fake',    'logOzone')

What would happen if we added this "fake" variable to the full model?

In [ ]:
fit_full_model = smf.ols(formula = 'logOzone ~ Temp + SolarR + Wind + fake', data=Air).fit()
print(fit_full_model.summary(slim=True))

The "slope" parameter for "fake" is not significant (not significantly different from zero). We see this from the p-value, which is larger than 0.05.

This is because "fake" *carries no information* - it is just random values and has no correlation with the outcome variable.

**Model Selection:**

So the question is how to choose which variable to include in a model - its not always better to include more variables. 

There are different approaches for selecting which explanatory variables to include in the model (*model selection*):

*Backward selection*: Remove the least significant variables one at the time

*Forward selection*: Start with the most significant variable and add one at the time

OBS: There is no perfect way to do model selection! One should also be aware of possible correlations in explanatory variables (collinearity). 

## Example: 3 treatment groups

In [ ]:
treatment_data = pd.read_csv("treatment_data.csv", sep=',')  # for this code to work, you need to save the treatment_data.csv file in the same folder as you keep this notebook
print(treatment_data)
print(len(treatment_data))

The variable "Treat" is qualitative and indicates which treatment group the patient belongs to. 

The variables Prewt and Postwt are is the weight of the patient before and after treatment. 

The variable Diff is simply (Postwt - Prewt)

OBS: The treatment is for patients with eating disorder - the goal of the treatment is to make patients gain weight

In [ ]:
# How many values in the categorical column (the column with text)?
print(treatment_data["Treat"].value_counts())

In [ ]:
# Visualise data (here we group by treatment type)

treatment_data.boxplot("diff", by='Treat', showmeans=True)

The mean values (of each group) are indicated by triangles.

### Compare two groups with **linear regression**:

In [ ]:
# Make sample data with only two of the treatment groups:
sample_data = treatment_data[treatment_data['Treat'].isin(["Cont", "FT"])]
print(len(sample_data))

In [ ]:
sample_data.boxplot("diff", by='Treat', showmeans=True)

We could of course compare these two groups with the methods we have learned for 2 samples. 

The comparison can be formulated as a linear regression problem:

In [ ]:
# Make a model with categorical variable:
sample_model = smf.ols(formula = 'diff ~ Treat', data=sample_data).fit()
print(sample_model.summary(slim=True))

Notice that the coefficient for FT treatment is 3.4962, which corresponds to the difference between the two means (see boxplot above).

Here the "Cont" group is chosen as a "reference group" and the effect of "FT" is shown in the model (Treat[T.FT] = 3.4962). This means that the average "diff" (i.e. the average weight-gain) in the FT group is 3.4962 higher than in the Cont group. 

### Multiple linear regression with categorical AND quantitative variables

We can also add Prewt in the regression model - maybe the weight-gain ("diff") also depends on the weight prior to treatment

In [ ]:
# Plot diff versus Prewt (pre-treatment weight)
# and indicate treatment group by color
colors = {'Cont':'black', 'CBT':'red', 'FT':'blue'}
treatment_data.plot.scatter("Prewt", "diff", c=treatment_data["Treat"].map(colors))


It seems that diff (weight-gain) is higher for the persons that had the lowest starting weight (smallest Prewt values).

The colors indicate the different treatment groups.

In [ ]:
# Model with categorical variabel (+ continous variabel):
cat_model = smf.ols(formula = 'diff ~ Prewt + Treat', data=treatment_data).fit()
print(cat_model.summary(slim=True))

In [ ]:
# Plot
x_values = np.arange(32,47,1)
y_values_cont = 22.5474 - 0.5655 * x_values - 1.8575
y_values_cbt  = 22.5474 - 0.5655 * x_values + 0
y_values_ft   = 22.5474 - 0.5655 * x_values + 2.0665

colors = {'Cont':'black', 'CBT':'red', 'FT':'blue'}
treatment_data.plot.scatter("Prewt", "diff", c=treatment_data["Treat"].map(colors))
plt.plot(x_values, y_values_cont, color="black")
plt.plot(x_values, y_values_cbt, color="red")
plt.plot(x_values, y_values_ft, color="blue")

plt.show()


This is a regression model with "Prewt" and "treatment group" as explanatory variables. 

The effect of Prewt is the same in all groups (the slope is the same in all groups, by design) 

The 3 groups have different "levels" 